# Implementation of the Newmark-β time-stepping

In this notebook, we present an  implementation of the Newmark-β time-stepping technique. We consider a cantilever beam of size `L × d` (2D, plane strain), fixed at its `x = 0` end, and subjected to a longitudinal or transversal load at `x = L` as in the precedent class session. 

![shema](setup_dynamics.png)

Introduction and elastodynamics equation
----------------------------------------

The elastodynamics equation combine the balance of linear momentum:

$$\nabla \cdot \sigma + \rho b = \rho \ddot{u}$$

where $u$ is the displacement vector field,
$\ddot{u}=\partial^2 u/\partial t^2$ is the acceleration, $\rho$ the
material density, $b$ a given body force and $\sigma$ the stress tensor
which is related to the displacement through a constitutive equation. In
the case of isotropic linearized elasticity, one has:

$$\sigma =\lambda \text{tr}(\varepsilon)\mathbb{1} + 2\mu\varepsilon$$

where $\varepsilon = (\nabla u + (\nabla u)^T)/2$ is the linearized
strain tensor, $\mathbb{1}$ is the identity of second-rank tensors and
$\lambda=\dfrac{E\nu}{(1+\nu)(1-2\nu)},\mu=\dfrac{E}{2(1+\nu)}$ are the
Lame coefficients given as functions of the Young modulus $E$ and the
Poisson ratio $\nu$.

The weak form is readily obtained by integrating by part the balance
equation using a test function $v\in V$ with $V$ being a suitable
function space that satisfies the displacement boundary conditions:

$$\int_{\Omega} \rho \ddot{u}\cdot v \, {\rm d} x + \int_{\Omega} \sigma(u):\varepsilon(v) \, {\rm d} x =
\int_{\Omega} \rho b \cdot v  \, {\rm d} x 
+ \int_{\partial\Omega} (\sigma\cdot n) \cdot v \, {\rm d} s \quad \text{for all } v \in V
$$

The previous equation can be written as follows:

$$
\text{Find }u\in V\text{ such that } m(\ddot{u},v) + k(u,v) = L(v) \quad \text{for all } v\in V
$$

where $m$ is the symmetric bilinear form associated with the mass matrix
and $k$ the one associated with the stiffness matrix.

After introducing the finite element space interpolation, one obtains
the corresponding discretized evolution equation:

$$\text{Find }\{u\}\in\mathbb{R}^n\text{ such that } \{v\}^T[M]\{\ddot{u}\} + \{v\}^T[K]\{u\} = \{v\}^T\{F\} \quad \text{for all } \{v\}\in\mathbb{R}^n$$

which is a generalized $n$-dof harmonic oscillator equation. 

Quite often in structural dynamics, structures do not oscillate
perfectly but lose energy through various dissipative mechanisms
(friction with air or supports, internal dissipation through plasticity,
damage, etc.). Dissipative terms can be introduced at the level of the
constitutive equation if these mechanisms are well known but quite often
it is not the case. Dissipation can then be modeled by adding an *ad
hoc* damping term depending on the structure velocity $\dot{u}$ to the
previous evolution equation:

$$\text{Find }u\in V\text{ such that } m(\ddot{u},v) + c(\dot{u},v) + k(u,v) = L(v) \quad \text{for all } v\in V$$

The damping form will be considered here as bilinear and symmetric,
being therefore associated with a damping matrix $[C]$.

### Rayleigh damping

When little is known about the origin of damping in the structure, a
popular choice for the damping matrix, known as *Rayleigh damping*,
consists in using a linear combination of the mass and stiffness matrix
$[C] = \eta_M[M]+\eta_K[K]$ with two positive parameters $\eta_M,\eta_K$
which can be fitted against experimental measures for instance (usually
by measuring the damping ratio of two natural modes of vibration).

In [ ]:
import dolfinx
import dolfinx.fem as fem
import numpy as np
import matplotlib.pyplot as plt
import ufl
import time
import os
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import dolfinx.fem.petsc
import pyvista
import utils

### Definition of the geometry and the mesh

In [ ]:
# geometry and mesh 
L = 1.0 # total length
d = 0.1*L # thickness
h = 0.2*d # size of a cell

my_domain = dolfinx.mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, -0.5*d), (L, 0.5*d)), n=(int(L/h), int(d/h)),
                            cell_type=dolfinx.mesh.CellType.triangle)

# Save the mesh in XDMF format
with dolfinx.io.XDMFFile(MPI.COMM_WORLD, "output_dyn_impl/mesh.xdmf", "w") as file:
    file.write_mesh(my_domain)
    my_domain.topology.create_connectivity(1, 2)

In [ ]:
pyvista.start_xvfb()
pyvista.set_jupyter_backend("static") # non-interactif, mais mieux
topology, cell_types, geometry_for_plotting = dolfinx.plot.vtk_mesh(my_domain, 2)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry_for_plotting)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.show_axes()
plotter.show_bounds(grid='front', location='outer', all_edges=True)
plotter.view_xy()

if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    pyvista.start_xvfb()
    figure = plotter.screenshot("fundamentals_mesh.png")

In [ ]:
# Define the different parts of the boundary
boundaries = [(1, lambda x: np.isclose(x[0], 0)),
              (2, lambda x: np.isclose(x[0], L)),
              (3, lambda x: np.isclose(x[1], -0.5*d)),
              (4, lambda x: np.isclose(x[1], 0.5*d))]

facet_indices, facet_markers = [], []
fdim = my_domain.topology.dim - 1
for (marker, locator) in boundaries:
    facets = dolfinx.mesh.locate_entities(my_domain, fdim, locator)
    facet_indices.append(facets) # here we put all the facets indices
    facet_markers.append(np.full_like(facets, marker)) # here we put all the facets 'labels' (1, 2, 3 or 4)
facet_indices = np.hstack(facet_indices).astype(np.int32) # concatenate everything in one big vector
facet_markers = np.hstack(facet_markers).astype(np.int32)
sorted_facets = np.argsort(facet_indices) # sorting
facet_tag = dolfinx.mesh.meshtags(my_domain, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

In [ ]:
ds = ufl.Measure("ds", domain=my_domain, subdomain_data=facet_tag)
dx = ufl.dx(domain=my_domain)

### Material parameters

In [ ]:
# Material parameters
Y  = 1.
nu = 0.3
mu = Y/(2.*(1.+nu))
lambda_ = Y*nu/((1.+nu)*(1.-2.*nu))
# lambda_ = 2*lambda_*mu/(lambda_+2*mu) # plane stress
rho = dolfinx.fem.Constant(my_domain,ScalarType(1.))

# Damping parameters
# The damping matrix C is defined as C = eta_m*M+eta_k*K
eta_m = dolfinx.fem.Constant(my_domain,ScalarType(0.01))
eta_k = dolfinx.fem.Constant(my_domain,ScalarType(0.00001))

### Definition of the problem in the linear (and in the nonlinear case with finite deformations)

In [ ]:
V = fem.functionspace(my_domain, ("Lagrange", 1, (my_domain.geometry.dim, )))

In [ ]:
boundary_clamped_dofs = dolfinx.fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1)) # dofs on the left end of the bar

#this is the boundary value of u(x,y) at the left border : it has to be zero (clamped)
u_D = np.array([0,0], dtype=ScalarType)

bc = dolfinx.fem.dirichletbc(u_D, boundary_clamped_dofs, V)

In [ ]:
I2 = ufl.Identity(my_domain.topology.dim)
# Kinematics
def strain_displacement(u):
    return ufl.sym(ufl.grad(u))

def stress_linear(eps):
    return lambda_*ufl.tr(eps)*I2+2.*mu*eps

def stiffness_linear(u, v):
    return ufl.inner(stress_linear(strain_displacement(u)),strain_displacement(v))*dx  

In [ ]:
traction = dolfinx.fem.Constant(my_domain, ScalarType((0,0)))

def mass(u, v):
    return rho*ufl.inner(u, v)*dx

def damping(u, v):
    return eta_m*mass(u, v) + eta_k*stiffness_linear(u, v)

def p_ext(v):
    return ufl.dot(traction,v)*ds(2) # traction is applied at right end of bar

In [ ]:
u = ufl.TrialFunction(V) # a generic unknown field
v = ufl.TestFunction(V) # a generic test field. This is _not_ the speed

In [ ]:
u_sol = dolfinx.fem.Function(V)# displacement of the solution
vit_sol = dolfinx.fem.Function(V)# velocity of the solution
acc_sol = dolfinx.fem.Function(V)# acceleration of the solution

### Parameters of the simulation¶

### Material, loading and time-discretization parameters

A time-dependent traction is applied at the $x = L$ face as follows
\begin{equation}
\vec T = \begin{cases}
\displaystyle\frac{t}{t_{\mathrm{c}}}\vec T_{\mathrm{max}} & t \leq t_{\mathrm{c}}\\[.2em]
\vec 0 & t > t_{\mathrm{c}}
\end{cases}
\end{equation}
where $t_{\mathrm{c}}$ is a “cut-off” time. Note that depending on the direction of the applied traction, we will need to select different values for `t_c` and `T_max`.

In [ ]:
def load_eval(t,t_cutoff):
    if t <= t_cutoff:
        return sigma_max*t/t_cutoff
    else:
        return 0

In [ ]:
stiffness = stiffness_linear
strain = strain_displacement

In [ ]:
# We compute the strain tensor of the solution
eps_solution = strain_displacement(u_sol)
V_eps = dolfinx.fem.functionspace(my_domain,("DG", 0))
eps_xx_expr = dolfinx.fem.Expression(eps_solution[0,0], V_eps.element.interpolation_points())
eps_xx = dolfinx.fem.Function(V_eps)

# `Questions start here`

Please fill the parts of the code marked by `XXX`

In [ ]:
eta_m.value = 0.01  # example eta_m = 0.01
eta_k.value = 0.00001 # example eta_k = 0.00001
# Loading
t_end = 10. # end time of the simulation 
# Is t_end large enough for flexion oscillations?
num_steps  = 500 # total number of steps for the entire simulation
dt = t_end/num_steps

t_cutoff = 1.95 # time duration for which we pull on the bar, this t_c
# Is t_cutoff large enough for loading the beam in flexion?
sigma_max = 0.01 # sigma_max

# Coordinates of the plotting point 
coords_tip = [L,0]

### Implicit time-stepping scheme

### Time discretization using the Newmark-$\beta$ method

We now introduce a time discretization of the interval study $[0;T]$ in
$N+1$ time increments $t_0=0,t_1,\ldots,t_N,t_{N+1}=T$ with
$\Delta t=T/N$ denoting the time step (supposed constant). The
resolution will make use of the Newmark-$\beta$ method in
structural dynamics. As an implicit method, it is unconditionally stable
for a proper choice of coefficients so that quite large time steps can
be used. It also allows for high frequency dissipation and offers a
second-order accuracy, i.e. in $O(\Delta t^2)$.

The method consists in solving the dynamic evolution equation at
intermediate time between $t_n$ and $t_{n+1}$ as follows:

$$[Eq1] \quad [M]\{\ddot{u}_{n+1}\} + [C]\{\dot{u}_{n+1}\}+[K]\{u_{n+1}\} = \{F(t_{n+1})\}$$

The following approximation for the displacement and velocity
at $t_{n+1}$ are used:

$$ [Eq2] \quad 
\{u_{n+1}\} = \{u_{n}\}+\Delta t \{\dot{u}_{n}\} + \dfrac{\Delta t^2}{2}\left((1-2\beta)\{\ddot{u}_{n}\}+2\beta\{\ddot{u}_{n+1}\}\right)
$$

$$ [Eq3] \quad
\{\dot{u}_{n+1}\} = \{\dot{u}_{n}\} + \Delta t\left((1-\gamma)\{\ddot{u}_{n}\}+\gamma\{\ddot{u}_{n+1}\}\right)
$$

### Time-discretization parameters

Parameters of the Newmark-β method.
A popular choice of parameters which ensures unconditional stability, optimal dissipation and second-order accuracy is:
$$\gamma=\frac{1}{2},\quad \beta=\frac{1}{4}$$

In [ ]:
gamma = 1./2.
beta = 1./4.

At time step $n$, when $u_n$, $\dot{u}_n$, and $\ddot{u}_n$ are known, [Eq1-2-3] can be used to write an equation for $\ddot{u}_{n+1}$ 

$$ [Eq4] \quad 
[A] \, \{\ddot{u}_{n+1}\} = \{F(t_{n+1})\} - [K] \, \{\hat{u}_n\} - [C] \, \{\hat{\dot{u}}_n\}
$$

with 
\begin{eqnarray}
[A] &=&  [M] + \gamma\Delta t\, [C] + \beta\Delta t^2\, [K] \\
\{\hat{u}_n\} &=& \{u_n\} + \Delta t \, \{\dot{u}_n\} + \frac{\Delta t^2}{2} \, (1-2\beta) \, \{\ddot{u}_n\} \\
\{\hat{\dot{u}}_n\} &=& \{\dot{u}_n\} + \Delta t \, (1-\gamma) \, \{\ddot{u}_n\}
\end{eqnarray}

Then, once [Eq4] is solved and we know $\{\ddot{u}_{n+1}\}$, we update the values of $\{u_n\}$ and $\{\dot{u}_n\}$ using [EQ2] and [EQ3]:
$$ 
[EQ5]\quad  \{u_{n+1}\} =  \{ \hat{u}_n\} + \Delta t^2 \, \beta \, \{\ddot{u}_{n+1}\}
$$
and
$$
[EQ6]\quad  \{\dot{u}_{n+1}\} =  \{ \hat{\dot{u}}_n\} + \Delta t \, \gamma \, \{\ddot{u}_{n+1}\}
$$

In [ ]:
# Definitions of operators, bilinear and linear forms

acc = ufl.TrialFunction(V) # unknown field for the acceleration

def A(acc, v): #this is used in EQ4
    return mass(acc, v) + XXX

f_load = p_ext(v) - XXX #this is used in EQ4

#this is EQ4
problem = dolfinx.fem.petsc.LinearProblem(A(acc,v), f_load, bcs=[bc], 
                                              petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

## Time discretization

In [ ]:
u_sol.x.array[:] = 0.
vit_sol.x.array[:] = 0.
acc_sol.x.array[:] = 0.
   
times = dt*np.arange(num_steps+1, dtype=np.float64)
displ_tip = np.zeros_like(times)
extension_strain = 0.0*times # Could we have written np.zeros_like(times)?

energies = np.zeros((num_steps+1, 4), dtype=np.float64)
E_damp = 0
E_elas = 0
E_ext = 0
E_kin = 0

with dolfinx.io.XDMFFile(my_domain.comm, "output_dyn_impl/time_steps_u.xdmf", "w") as file:
    file.write_mesh(my_domain)

with dolfinx.io.XDMFFile(my_domain.comm, "output_dyn_impl/elastodynamicsresults_stress.xdmf", "w") as file_stress:
    file_stress.write_mesh(my_domain)
    
time_solve = 0.

for n in range(num_steps):
    if (n%100 == 0):
        print(n,' / ', num_steps)

    # 1. Predictor step
    # here you have to define hat(u) and hat(dot(u))
    # you can use U.vector.axpy(b,V) : this does U=U+bV
    # where U and V are vectors and b is a scalar
    u_XXX.vector.axpy(XXX)
    vit_XXX.vector.axpy(XXX)

    # Update the traction value : F(t_{n+1})
    traction.value = XXX
    
    # 2. Update acceleration
    # Solve with  A acc = ...
    XXX = problem.solve()
    
    # 3. Corrector step
    # here we have to update u_sol and vit_sol, using EQ5 and EQ6
    u_XXX.vector.axpy(XXX, XXX)
    vit_XXX.vector.axpy(XXX, acc_sol.vector)
    
    # Postprocessing 
    displ_tip[n+1] = utils.evaluate_at_points(np.array([[coords_tip[0]], [coords_tip[1]], [0]]), u_sol)[0,0]
    E_elas = dolfinx.fem.assemble_scalar(dolfinx.fem.form(0.5*stiffness(u_sol, u_sol)))
    E_kin = dolfinx.fem.assemble_scalar(dolfinx.fem.form(0.5*mass(vit_sol, vit_sol)))
    E_damp += dt*dolfinx.fem.assemble_scalar(dolfinx.fem.form(damping(vit_sol, vit_sol)))
    E_tot = E_elas+E_kin+E_damp
    energies[n+1, :] = np.array([E_elas, E_kin, E_damp, E_tot])

    # Stress computation
    eps_xx.interpolate(eps_xx_expr)
    extension_strain[n+1] = utils.evaluate_at_points(np.array([[0.5*L], [0.25*d], [0]]), eps_xx)[0,0]

    if (n%10 == 0):
        file.write_function(u_sol, (n+1)*dt)
        file_stress.write_function(eps_xx, (n+1)*dt)

# Close xmdf file
file.close()
file_stress.close()
print('Total time for Lin. Alg. solving:',time_solve)
print('Total time for file saving:',XXX)
print('Total time for stress interpolation:',XXX)
print('Interesting other total time (what for?):',XXX)

In [ ]:
plt.plot(times, extension_strain, '.-')
plt.xlabel("Time")
plt.ylabel("eps_xx(0.5L ; 0.25 d)")
plt.show()

In [ ]:
plt.figure()
plt.plot(times, displ_tip, '.-')
plt.plot(times, sigma_max*L/Y*np.ones_like(times), '--g') # what is this limit?
plt.plot(times, -sigma_max*L/Y*np.ones_like(times), '--g') # what is this limit?
plt.grid()
plt.xlabel("Time")
plt.ylabel("Tip displacement")
#plt.savefig("tip_displacement.png")

In [ ]:
plt.figure()
plt.plot(times, energies)
plt.legend(("elastic", "kinetic", "damping", "total"))
plt.xlabel("Time")
plt.ylabel("Energies")
plt.show()

# Questions

Here, we do not consider any physical disspation (eta_m=0, eta_k=0).

1 - We set E=1, nu=0.3, Lx=1, Ly=0.1, rho=1. What is the period of the oscillations? What kind of oscillations is this? (Use paraview to decide). How does this period change if we (i) divide Lx by 2? (ii) multiply Lx by 2? Compare with the analytical value for a bar in extension/compression.

1.5 With same parameter values (E=1, nu=0.3, Lx=1, Ly=0.1, rho=1), solve the dynamics when the external load is vertical. What kind of oscillations is this? (Use paraview to decide). How does this period change if we (i) divide Lx by 2? (ii) multiply Lx by 2? Compare with the analytical value for an Euler-Bernoulli beam in flexural vibrations.

2 - Compare the value of the current time step with the critical time step of the Courant condition of the explicit method (dt_crit = rmin/np.sqrt(E/rho.values()) for the Courant condition)

3 - Test the case $\gamma < \frac{1}{2}$.

4 - Test the case $\gamma > \frac{1}{2}$ and $2\beta > \gamma$. What is happening?

4.5 - In the plane $(\gamma,\beta)$, find the regions where the algorithm is unstable, conditionally stable, and unconditionaly stable. Stay in the region where $\gamma<1$ and $\beta<0.5$.

5 - What happens for low values of $t_c$ like $t_c=0.02$? What to change to have a program that works correctly? Which strategy to adopt for very small values of $t_c$?

6 - Compare the time taken by the computer to (i) solve the linear algebra system at each time step, (ii) write the output files, (iii) execute the rest of the (time-step) loop. Where does the compute spend most time? Which dolfin function is time-consuming? Does this change with parameters?

7 - Write up a python function that perfoms time integration of the system and exports xdmf files to be open with Paraview. The function will take inputs such as Young's modulus, size of the system, time-step, etc.